# Simon's algorithm

Simon's algorithm is closely related in principle to the Bernstein-Vazirani algorithm and the Deutsch-Josza algorithm, (which are also explained in other notebooks in this repository).

We are given an "oracle" and promised that its outputs are either **one-to-one** or **two-to-one**, i.e., it either has **exactly** as many possible outputs as possible inputs or **exactly half** as many possible outputs as possible inputs. In effect, all two-to-one functions can be thought of as being based on a "**hidden string**" encoded into the oracle, whereas one-to-one functions are the only case where the hidden string is "**0**." Our problem is therefore to determine the "hidden string" encoded into the oracle.

A classical computer, on average, needs to check about half the possible inputs to answer the question of what the hidden string is; a quantum computer needs **exponentially fewer** queries of the oracle to determine the same answer.

(...So the conventional disciplinary thinking goes, at least. But notice that we are doing 120 qubits of Simon's algorithm below, and we could argue we only need a single query, with PyQrack!)

In [1]:
import os
import random
from collections import Counter
from pyqrack import QrackSimulator

# Tell Qrack that it can use as many stabilizer qubits as it wants,
# without worrying about converting to ket for GPU:
os.environ['QRACK_MAX_CPU_QB']='-1'

# Range is up to 2 ** input_size - 1
input_size = 80

# Arbitrary value within range:
hidden_bits = 2**40 - 1285

num_qubits = input_size * 2
oracle_qubits = [*range(input_size)]

oracle_swap_tuples = []
for i in range(input_size // 2):
    i = random.randint(input_size, (input_size * 2) - 1)
    j = random.randint(input_size, (input_size * 2) - 1)
    if i != j:
        oracle_swap_tuples.append((i, j))

You may change the `hidden_bits` above to be any value that can fit within the oracle register width. Your `hidden_bits` corresponds to the oracle below:

In [2]:
def oracle(sim):
    # The first portion of this oracle is effectively a one-to-one oracle,
    # in itself, (though not the only possible one-to-one oracle).
    for i in oracle_qubits:
        sim.mcx([i], i + input_size)

    # The second portion of this oracle makes it two-to-one,
    # (unless "hidden_bits" is 0):
    for i in oracle_qubits:
        # We pick any one of the "set" hidden bits:
        if (hidden_bits >> i) & 1:
            for j in oracle_qubits:
                # Act CNOT between the "set" hidden bit in input and all hidden bits in output:
                if (hidden_bits >> j) & 1:
                    sim.mcx([i], j + input_size)
            # One bit in the "i" loop is sufficient:
            break;

    # We can now swap at random in the output register
    # and still satisfy our promises about the oracle:
    for i in oracle_swap_tuples:
         sim.swap(i[0], i[1])

This is the algorithm itself. (A classical simulator of quantum computers can run the "unitary preamble" exactly once and then secondarily "sample" the measurement distribution of the end state.)

In [3]:
# Prepare the initial register state:
sim = QrackSimulator(num_qubits, isCpuGpuHybrid=False, isOpenCL=False)
for i in oracle_qubits:
    sim.h(i)

# Make exactly one query to the oracle:
oracle(sim)

# Finish the unitary portion of the algorithm, with the result from the oracle...
# (It is often presented that measurement of the second register needs to occur before this step;
# this is actually wholly theoretically unnecessary in the ideal, due to locality of quantum information.)
for i in oracle_qubits:
    sim.h(i)

# The cost in "shot" count is linear, at this point.
# results = Counter(sim.measure_shots(oracle_qubits, 32768))

# The shared library API definition prevents high-width sampling, above a limit.
results = {}
for i in range(32768):
    s = QrackSimulator(cloneSid = sim.sid)
    s.m_all()
    result = 0
    for j in oracle_qubits:
        if s.m(j):
            result |= 1 << j
    if result in results:
        results[result] = results[result] + 1
    else:
        results[result] = 1

The ultimate point of the algorithm is that, by solving the system of equations below, **where the value of `hidden_string` is an algebraic unknown**, we can determine the value of `hidden_string`, with a probabilistic likelihood that converges exponentially faster than a "classical" algorithm, in the number of measurement "shots." (In a real hardware quantum computer, the number of "shots" would correspond to exactly the number of queries of the oracle).

Let's check if our implied system of equations is right:

In [4]:
for key in results.keys():
    x = (bin(hidden_bits & key).count("1")) & 1
    if x > 0:
        print("Failed!", hidden_bits, ".", key, "=", x, "(mod 2)")
        break
    # print(hidden_bits, ".", key, "=", x, "(mod 2)")
print("Done checking output!")

Done checking output!


To be fair, Qrack defines a general one-to-one function, with the aid of "QRAM," which might or might not be reducible to a Clifford circuit: we call this operation `hash`, in PyQrack.

In [5]:
# Range is up to 2 ** input_size - 1
input_size = 8

# Arbitrary value within range:
hidden_bits = 120

num_qubits = input_size * 2
oracle_qubits = [*range(input_size)]
output_qubits = [*range(input_size, 2 * input_size)]

oracle_swap_tuples = []
for i in range(input_size // 2):
    i = random.randint(input_size, (input_size * 2) - 1)
    j = random.randint(input_size, (input_size * 2) - 1)
    if i != j:
        oracle_swap_tuples.append((i, j))

# Notice that the RAM cost is exponential in qubit count:
hash_map = list(range(0, 256))
random.shuffle(hash_map)

def hash_oracle(sim):
    # The first portion of this oracle is a general one-to-one oracle, in itself.
    # First, copy the input register into the output register:
    for i in oracle_qubits:
        sim.mcx([i], i + input_size)
    # Next, replace each input value with its hash map value:
    sim.hash(output_qubits, hash_map)

    # The second portion of this oracle makes it two-to-one,
    # (unless "hidden_bits" is 0):
    for i in oracle_qubits:
        # We pick any one of the "set" hidden bits:
        if (hidden_bits >> i) & 1:
            for j in oracle_qubits:
                # Act CNOT between the "set" hidden bit in input and all hidden bits in output:
                if (hidden_bits >> j) & 1:
                    sim.mcx([i], j + input_size)
            # One bit in the "i" loop is sufficient:
            break;

    # We can now swap at random in the output register
    # and still satisfy our promises about the oracle:
    for i in oracle_swap_tuples:
         sim.swap(i[0], i[1])

In [6]:
# Prepare the initial register state:
sim = QrackSimulator(num_qubits, isCpuGpuHybrid=False, isOpenCL=False)
for i in oracle_qubits:
    sim.h(i)

# Make exactly one query to the oracle:
oracle(sim)

# Finish the unitary portion of the algorithm, with the result from the oracle...
# (It is often presented that measurement of the second register needs to occur before this step;
# this is actually wholly theoretically unnecessary in the ideal, due to locality of quantum information.)
for i in oracle_qubits:
    sim.h(i)

# The cost in "shot" count is linear, at this point.
# results = Counter(sim.measure_shots(oracle_qubits, 32768))

# The shared library API definition prevents high-width sampling, above a limit.
results = {}
for i in range(32768):
    s = QrackSimulator(cloneSid = sim.sid)
    s.m_all()
    result = 0
    for j in oracle_qubits:
        if s.m(j):
            result |= 1 << j
    if result in results:
        results[result] = results[result] + 1
    else:
        results[result] = 1

In [7]:
for key in results.keys():
    x = (bin(hidden_bits & key).count("1")) & 1
    if x > 0:
        print("Failed!", hidden_bits, ".", key, "=", x, "(mod 2)")
        break
    print(hidden_bits, ".", key, "=", x, "(mod 2)")

120 . 231 = 0 (mod 2)
120 . 227 = 0 (mod 2)
120 . 122 = 0 (mod 2)
120 . 3 = 0 (mod 2)
120 . 203 = 0 (mod 2)
120 . 174 = 0 (mod 2)
120 . 201 = 0 (mod 2)
120 . 155 = 0 (mod 2)
120 . 48 = 0 (mod 2)
120 . 248 = 0 (mod 2)
120 . 102 = 0 (mod 2)
120 . 83 = 0 (mod 2)
120 . 4 = 0 (mod 2)
120 . 204 = 0 (mod 2)
120 . 225 = 0 (mod 2)
120 . 153 = 0 (mod 2)
120 . 7 = 0 (mod 2)
120 . 100 = 0 (mod 2)
120 . 132 = 0 (mod 2)
120 . 177 = 0 (mod 2)
120 . 44 = 0 (mod 2)
120 . 82 = 0 (mod 2)
120 . 43 = 0 (mod 2)
120 . 215 = 0 (mod 2)
120 . 97 = 0 (mod 2)
120 . 202 = 0 (mod 2)
120 . 80 = 0 (mod 2)
120 . 103 = 0 (mod 2)
120 . 5 = 0 (mod 2)
120 . 42 = 0 (mod 2)
120 . 96 = 0 (mod 2)
120 . 157 = 0 (mod 2)
120 . 46 = 0 (mod 2)
120 . 1 = 0 (mod 2)
120 . 226 = 0 (mod 2)
120 . 49 = 0 (mod 2)
120 . 176 = 0 (mod 2)
120 . 251 = 0 (mod 2)
120 . 76 = 0 (mod 2)
120 . 205 = 0 (mod 2)
120 . 210 = 0 (mod 2)
120 . 152 = 0 (mod 2)
120 . 54 = 0 (mod 2)
120 . 159 = 0 (mod 2)
120 . 130 = 0 (mod 2)
120 . 120 = 0 (mod 2)
120 . 101 =

However, as we note in the code comments, the cost to encode the oracle itself becomes **exponential**. Whether this operation can be reduced to a Clifford circuit, the exponential cost of encoding seems like it might defeat the point of the argument from Simon's algorithm, for an oracular separation between BQP and BPP.